In [ ]:
import logging
import warnings

import pandas as pd
from lightning import seed_everything

from fxguard.data import TimeSeries
from fxguard.models import TiDELite
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from darts.utils.model_selection import train_test_split
from fxguard.utils.scaler import Scaler
warnings.filterwarnings('ignore', 'GPU available')
logging.getLogger('lightning.pytorch.accelerators.cuda').setLevel(logging.WARNING)


SEED = seed_everything(42)

In [ ]:
df = pd.read_csv('../data/raw/yfinance_source.csv', parse_dates=['date'])

In [ ]:
melt = df.melt(id_vars='date').dropna()
melt['variable'] = melt['variable'].astype('category').cat.codes # here to try get static covs to be used. Encoding to be moved inside dataset after
ts = TimeSeries.from_group_dataframe(melt, group_col='variable', time_col='date', value_cols='value')

In [ ]:
scaler = Scaler()
ts_scaled = scaler.fit_transform(ts)

In [ ]:
temp = []
for _ in range(len(ts[5])//5):
    temp.extend([1,2,3,4,5])

past_cov = TimeSeries(times=ts_scaled[5].time_index,values=temp,components=['test'])
covariates = {'past_covariates':None,'future_covariates':None}

In [ ]:
model = TiDELite(20, 1,trainer_kwargs={'accelerator':'cpu','deterministic':True},batch_size=128,use_static_covariates=False)
model.fit(ts_scaled,**covariates,epochs=2)

In [ ]:
forecast = model.historical_forecast(ts_scaled,**covariates)

In [ ]:
ax = forecast[0].plot(label='predicted')
ts_scaled[0].plot(ax=ax)

In [ ]:
from darts import TimeSeries
from darts.models import TiDEModel
from darts.dataprocessing.transformers import Scaler
d = df.copy().set_index('date',drop=True).asfreq('D').ffill()
scaler = Scaler()

In [ ]:
d_melt = d.reset_index().melt(id_vars='date').dropna().set_index('date')
d_melt['variable'] = d_melt['variable'].astype('category').cat.codes
data = TimeSeries.from_group_dataframe(d_melt,group_cols='variable',value_cols='value')
data = scaler.fit_transform(data)
d_model = TiDEModel(20,1,pl_trainer_kwargs={'max_steps':1000,'accelerator':'cpu','deterministic':True},batch_size=128,use_static_covariates=False)
d_model.fit(data)
d_forecast = d_model.historical_forecasts(series=data,retrain=False,last_points_only=True)
d_forecast[0].plot()
data[0].plot()